In [ ]:
pow(3.5 - 4, 2)

0.25

In [ ]:
from math import sqrt

# Calculando a distancia entre Claudia e Ana
distance = sqrt(pow(3 - 3, 2) + pow(3.5 - 4, 2))
print('Distância/Similaridade entre Claúdia e Ana: ', distance)

# Convertendo para percentual em uma escala de 0 e 1
percentDistance = 1/(1+distance)
print('Distância/Similaridade entre Cláudia e Ana - Percentual : ', percentDistance)

Distância/Similaridade entre Claúdia e Ana:  0.5
Distância/Similaridade entre Cláudia e Ana - Percentual :  0.6666666666666666


In [ ]:
# Calculando a distancia entre Marcos e Ana
distance = sqrt(pow(3 - 1.5, 2) + pow(3.5 - 5, 2))
print('Distância/Similaridade entre Marcos e Ana: ', distance)

# Convertendo para percentual em uma escala de 0 e 1
percentDistance = 1/(1+distance)
print('Distância/Similaridade entre Marcos e Ana - Percentual : ', percentDistance)

Distância/Similaridade entre Marcos e Ana:  2.1213203435596424
Distância/Similaridade entre Marcos e Ana - Percentual :  0.32037724101704074


### Dataset Usuários e Notas

In [ ]:
avaliacoesUsuarios = {'Ana': 
		{'Freddy x Jason': 2.5, 
		 'O Ultimato Bourne': 3.5,
		 'Star Trek': 3.0, 
		 'Exterminador do Futuro': 3.5, 
		 'Norbit': 2.5, 
		 'Star Wars': 3.0},
	 
	  'Marcos': 
		{'Freddy x Jason': 3.0, 
		 'O Ultimato Bourne': 3.5, 
		 'Star Trek': 1.5, 
		 'Exterminador do Futuro': 5.0, 
		 'Star Wars': 3.0, 
		 'Norbit': 3.5}, 

	  'Pedro': 
	    {'Freddy x Jason': 2.5, 
		 'O Ultimato Bourne': 3.0,
		 'Exterminador do Futuro': 3.5, 
		 'Star Wars': 4.0},
			 
	  'Claudia': 
		{'O Ultimato Bourne': 3.5, 
		 'Star Trek': 3.0,
		 'Star Wars': 4.5, 
		 'Exterminador do Futuro': 4.0, 
		 'Norbit': 2.5},
				 
	  'Adriano': 
		{'Freddy x Jason': 3.0, 
		 'O Ultimato Bourne': 4.0, 
		 'Star Trek': 2.0, 
		 'Exterminador do Futuro': 3.0, 
		 'Star Wars': 3.0,
		 'Norbit': 2.0}, 

	  'Janaina': 
	     {'Freddy x Jason': 3.0, 
	      'O Ultimato Bourne': 4.0,
	      'Star Wars': 3.0, 
	      'Exterminador do Futuro': 5.0, 
	      'Norbit': 3.5},
			  
	  'Leonardo': 
	    {'O Ultimato Bourne':4.5,
             'Norbit':1.0,
	     'Exterminador do Futuro':4.0}
}

### Função Euclidiana

In [ ]:
from math import sqrt

def euclidiana(dataset, usuario1, usuario2):
  si = {}
  for item in dataset[usuario1]:
    if item in dataset[usuario2]:
      si[item] = 1

  if len(si) == 0:
    return 0

  soma = sum([pow(dataset[usuario1][item] - dataset[usuario2][item], 2)
              for item in dataset[usuario1]
                if item in dataset[usuario2]])
  
  return 1/(1+sqrt(soma))

### Similaridade entre Leonardo e Ana

In [ ]:
euclidiana(avaliacoesUsuarios, 'Leonardo', 'Ana')

0.3483314773547883

### Similaridade entre Marcos e Cláudia

In [ ]:
euclidiana(avaliacoesUsuarios, 'Marcos', 'Claudia')

0.28172904669025317

### Similiaridade entre Pedro e Marcos

In [ ]:
euclidiana(avaliacoesUsuarios, 'Pedro', 'Marcos')

0.3405424265831667

# Recomendação a partir de usuários similares

## Criando Função que retorna todos os usuario similares com o usuario informado no parâmetro


In [ ]:
# Retorna os usuários e percentual de similaridade, dos 30 primeiros usuários mais similares
def getSimilares(dataset, usuario):
  similaridade = [(euclidiana(dataset, usuario, outro), outro)
                  for outro in dataset
                    if outro != usuario]

  similaridade.sort()
  similaridade.reverse()

  return similaridade[0:30]

### Verificando as similaridades do usuario Ana

In [ ]:
# O usuário mais similar com a Ana é o Pedro com 47%
getSimilares(avaliacoesUsuarios, 'Ana')

[(0.4721359549995794, 'Pedro'),
 (0.4142135623730951, 'Adriano'),
 (0.38742588672279304, 'Claudia'),
 (0.3483314773547883, 'Leonardo'),
 (0.3405424265831667, 'Janaina'),
 (0.29429805508554946, 'Marcos')]

## Recomendação baseada em usuários

### Função para obter as recomendações

In [ ]:
# 1 - Calcular a similaridade do usuário alvo com todos os demais
# 2 - Multiplicar a similaridade pela nota dos demais usuários para cada filme que o usuário alvo não assistiu
# 3 - Somar a similaridade dos demais usuários em relação ao usuário alvo para cada filme que os demais usuários deram nota e o usuário alvo não assistiu
# 4 - Dividir o total de pesos(similaridade x nota) pelo total de similaridade
# 5 - Retorna a lista de filmes(30 primeiros) que o usuário alvo não assistiu e as notas previstas que ele daria para cada filme

def getRecomendacoesUsuario(dataset, usuario):
  totais = {}
  somaSimilaridade = {}

  for outro in dataset:
    if outro == usuario:
      continue
    else:
      similaridade = euclidiana(dataset, usuario, outro)
      if similaridade <= 0: 
        continue
      for item in dataset[outro]:
        if item not in dataset[usuario]:
          totais.setdefault(item, 0)
          totais[item] += dataset[outro][item] * similaridade
          somaSimilaridade.setdefault(item, 0)
          somaSimilaridade[item] += similaridade

  rankings = [(total / somaSimilaridade[item], item) for item, total in totais.items()]
  
  rankings.sort()
  rankings.reverse()

  return rankings[0:30]

### Obtendo as recomendações de filmes que o Leonardo não assistiu

In [ ]:
getRecomendacoesUsuario(avaliacoesUsuarios, 'Leonardo')

[(3.457128694491423, 'Star Wars'),
 (2.778584003814924, 'Freddy x Jason'),
 (2.422482042361917, 'Star Trek')]

### Obtendo as recomendações de filmes que o Pedro não assistiu

In [ ]:
getRecomendacoesUsuario(avaliacoesUsuarios, 'Pedro')

[(2.4825817086405517, 'Star Trek'), (2.453379230569188, 'Norbit')]

# Recomendação á partir de Filmes Similares

### Base de dados invertida

In [ ]:
avaliacoesFilmes = {'Freddy x Jason': 
		{'Ana': 2.5, 
		 'Marcos:': 3.0 ,
		 'Pedro': 2.5, 
		 'Adriano': 3.0, 
		 'Janaina': 3.0 },
	 
	 'O Ultimato Bourne': 
		{'Ana': 3.5, 
		 'Marcos': 3.5,
		 'Pedro': 3.0, 
		 'Claudia': 3.5, 
		 'Adriano': 4.0, 
		 'Janaina': 4.0,
		 'Leonardo': 4.5 },
				 
	 'Star Trek': 
		{'Ana': 3.0, 
		 'Marcos:': 1.5,
		 'Claudia': 3.0, 
		 'Adriano': 2.0 },
	
	 'Exterminador do Futuro': 
		{'Ana': 3.5, 
		 'Marcos:': 5.0 ,
		 'Pedro': 3.5, 
		 'Claudia': 4.0, 
		 'Adriano': 3.0, 
		 'Janaina': 5.0,
		 'Leonardo': 4.0},
				 
	 'Norbit': 
		{'Ana': 2.5, 
		 'Marcos:': 3.0 ,
		 'Claudia': 2.5, 
		 'Adriano': 2.0, 
		 'Janaina': 3.5,
		 'Leonardo': 1.0},
				 
	 'Star Wars': 
		{'Ana': 3.0, 
		 'Marcos:': 3.5,
		 'Pedro': 4.0, 
		 'Claudia': 4.5, 
		 'Adriano': 3.0, 
		 'Janaina': 3.0}
}

### Retorna os filmes mais similares em relação á nota

In [ ]:
getSimilares(avaliacoesFilmes, 'Star Wars')

[(0.3761785115301142, 'Freddy x Jason'),
 (0.3266316347104093, 'O Ultimato Bourne'),
 (0.29429805508554946, 'Norbit'),
 (0.27429188517743175, 'Exterminador do Futuro'),
 (0.2708131845707603, 'Star Trek')]

### Retorna pessoas que assistiram e deram notas para o filme informado

Dessa forma, poderia ser enviado para essas pessoas, de repente um ingresso promocional para a pré estréia de um filme relacionado ou continuação do mesmo


In [ ]:
getRecomendacoesUsuario(avaliacoesFilmes, 'Star Wars')

[(3.4999999999999996, 'Marcos'), (3.196200842319029, 'Leonardo')]

## Trabalhando com a base de dados Movielens

In [ ]:
import pandas as pd

# Carregando o dataset de filmes(arquivo u.item)
dfMovies = pd.read_csv('https://dl.dropboxusercontent.com/s/2yyed2vbe9zt6j4/u.item.csv?dl=0', sep="|", header=None, encoding ="ISO-8859-1")

# Carregando o dataset de avaliação dos filmes(arquivo u.data)
dfStars = pd.read_csv('https://dl.dropboxusercontent.com/s/fwdvlf13c3ob8h0/u.data?dl=0', sep="\t", header=None, encoding="ISO-8859-1")

# Carrega um dicionario estruturado com o usuario, os filmes que o mesmo avaliou e a nota que foi dada para cada um dos filmes 
def carregaMovieLens():
  
  filmes = {}
  for index, row in dfMovies.iterrows():
    (id, titulo) = row[0:2]
    filmes[id] = titulo

  base = {}
  for index, row in dfStars.iterrows():
    (id_usuario, id_filme, nota, tempo) = row[0:4]
    base.setdefault(id_usuario, {})
    base[id_usuario][filmes[id_filme]] = float(nota)

  return base

In [ ]:
base = carregaMovieLens()

In [ ]:
# Retorna a lista de usuários mais similares e o percentual de similaridade(0 a 1) 
getSimilares(base, 212)

[(1.0, 915),
 (1.0, 866),
 (1.0, 856),
 (1.0, 850),
 (1.0, 849),
 (1.0, 837),
 (1.0, 820),
 (1.0, 818),
 (1.0, 809),
 (1.0, 808),
 (1.0, 801),
 (1.0, 770),
 (1.0, 759),
 (1.0, 722),
 (1.0, 612),
 (1.0, 596),
 (1.0, 584),
 (1.0, 583),
 (1.0, 580),
 (1.0, 575),
 (1.0, 572),
 (1.0, 547),
 (1.0, 544),
 (1.0, 529),
 (1.0, 519),
 (1.0, 478),
 (1.0, 461),
 (1.0, 431),
 (1.0, 427),
 (1.0, 412)]

In [ ]:
# Retorna os 30 primeiros filmes que o usuario não assistiu e os filmes para os quais o sistema previu que ele daria as notas mais altas
getRecomendacoesUsuario(base, 212)

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Aiqing wansui (1994)'),
 (4.678464513946264, 'Delta of Venus (1994)'),
 (4.669110773445877, 'Innocents, The (1961)'),
 (4.6679881166781785, 'Pather Panchali (1955)'),
 (4.643210827674669, 'Letter From Death Row, A (1998)'),
 (4.636191497550252, 'Maya Lin: A Strong Clear Vision (1994)'),
 (4.6151289598408765, 'Anna (1996)'),
 (4.558310593401602, "Some Mother's Son (1996)"),
 (4.493969532782648, 'Shawshank Redemption, The (1994)'),
 (4.481115124220606, 'Leading Man, The (1996)'),
 (4.45837363085848, 'For the Moment (1994)'),
 (4.457258689756284, 'Third Man, The (1949)'),
 (4.445996248823582, 'Casablanca (1

## Recomendação de filmes baseada em itens

### Passo a passo do algoritmo de recomendação de filmes baseado em itens

Para recomendar um filme para um usuário, no exemplo abaixo, Leonardo, iremos seguir os seguintes passos:

1 - Identificar todos os filmes que ele assistiu.

![alt text](https://dl.dropboxusercontent.com/s/0w96ggcmrvkheal/0001.png?dl=0)

2 - Identificar todos os filmes que ele não assistiu.

![alt text](https://dl.dropboxusercontent.com/s/tqo8obw5f8w4rxw/0002.PNG?dl=0)

3 - Utilizando a já criada função getSimilares, iremos obter o percentual de similaridade que possui os filmes NÃO ASSISTIDOS em relação aos JÁ ASSISTIDOS.

![alt text](https://dl.dropboxusercontent.com/s/9ooima1uqdmziqb/0003.png?dl=0)

4 - Calcular a nota de similaridade de cada filme NÃO ASSISTIDO versus cada filme JÁ ASSISTIDO. 
<br>Fórmula: Nota do filme já assistido x o percentual de similaridade do filme não assistido

![alt text](https://dl.dropboxusercontent.com/s/n0z3l1ujnlv63qu/0005.PNG?dl=0)

5 - Fazer o somatório da coluna de percentual de similaridade de cada filme NÃO ASSISTIDO.

![alt text](https://dl.dropboxusercontent.com/s/w60xt11qg9kwqwk/0006.PNG?dl=0)

6 - Fazer o somatório da coluna de nota de similaridade de cada filme NÃO ASSISTIDO.

![alt text](https://dl.dropboxusercontent.com/s/w3gvgfl66usqtrq/0007.PNG?dl=0)

7 - Por fim, para cada filme NÃO ASSISTIDO, é calculado a NOTA ESTIMIDA do usuário Leonardo para cada filme NÃO ASSISTIDO: 
<br>Fórmula: somatória da nota de similaridade / somatória do percentual de similaridade

![alt text](https://dl.dropboxusercontent.com/s/7frub5joynzscj2/0008.PNG?dl=0)









In [ ]:
# Calculando a similaridade de cada filme com todos os outros
def calculaItensSimilares(base):
  result = {}

  for item in base:
    notas = getSimilares(base, item)
    result[item] = notas
  
  return result

Em um ambiente de produção, rodaríamos a função acima periodicamente(1 x por dia por exemplo) e carregaríamos os dados retornados em uma base de dados para as consultas. No exemplo abaixo, estamos carregando apenas em uma variável, denominada **itensSimilares** que será utilizada nos passos posteriores. 

In [ ]:
# Carregando a lista de similaridades entre os filmes
itensSimilares = calculaItensSimilares(avaliacoesFilmes)

In [ ]:
itensSimilares

{'Exterminador do Futuro': [(0.3761785115301142, 'O Ultimato Bourne'),
  (0.27429188517743175, 'Star Wars'),
  (0.2402530733520421, 'Freddy x Jason'),
  (0.20799159651347807, 'Star Trek'),
  (0.18464218557642828, 'Norbit')],
 'Freddy x Jason': [(0.4721359549995794, 'Norbit'),
  (0.3761785115301142, 'Star Wars'),
  (0.3567891723253309, 'O Ultimato Bourne'),
  (0.3483314773547883, 'Star Trek'),
  (0.2402530733520421, 'Exterminador do Futuro')],
 'Norbit': [(0.4721359549995794, 'Freddy x Jason'),
  (0.3761785115301142, 'Star Trek'),
  (0.29429805508554946, 'Star Wars'),
  (0.1886378647726465, 'O Ultimato Bourne'),
  (0.18464218557642828, 'Exterminador do Futuro')],
 'O Ultimato Bourne': [(0.3761785115301142, 'Exterminador do Futuro'),
  (0.3567891723253309, 'Freddy x Jason'),
  (0.3266316347104093, 'Star Wars'),
  (0.32037724101704074, 'Star Trek'),
  (0.1886378647726465, 'Norbit')],
 'Star Trek': [(0.3761785115301142, 'Norbit'),
  (0.3483314773547883, 'Freddy x Jason'),
  (0.320377241017

In [ ]:
# Função que dado um usuário e uma lista de similiaridades entre os filmes, retorna uma lista de itens/filmes 
# que o usuário não assistiu ordenado/rankeado por ordem dos filmes que o usuário daria a maior nota
# Primeiro parâmetro - base de dados contendo os usuários e as notas que eles deram para cada filme assistido
# Segundo parâmetro - lista de similaridade entre os itens
# Terceiro parâmetro - O nome do usuário para o qual se deseja obter a recomendação de filmes não assistidos e que possivelmente ele mais vai gostar
def getRecomendacoesItens(baseUsuario, similaridadeItens, usuario):
  
  notasUsuario = baseUsuario[usuario]
  notas = {}
  totalSimilaridade = {}

  for (item, nota) in notasUsuario.items():
    for (similaridade, item2) in similaridadeItens[item]:
      if item2 in notasUsuario: continue
      notas.setdefault(item2, 0)
      notas[item2] += similaridade * nota
      totalSimilaridade.setdefault(item2, 0)
      totalSimilaridade[item2] += similaridade

  rankings = [(score/totalSimilaridade[item], item) for item, score in notas.items()]
  rankings.sort()
  rankings.reverse()

  return rankings

## Testando as duas abordagens de recomendação para o usuário Leonardo
#### Recomendação baseada em usuário

In [ ]:
getRecomendacoesUsuario(avaliacoesUsuarios, 'Leonardo')

[(3.457128694491423, 'Star Wars'),
 (2.778584003814924, 'Freddy x Jason'),
 (2.422482042361917, 'Star Trek')]

#### Recomendação baseada em itens/filmes

In [ ]:
getRecomendacoesItens(avaliacoesUsuarios, itensSimilares, 'Leonardo')

[(3.1962008423190285, 'Star Wars'),
 (2.929467965289551, 'Star Trek'),
 (2.8420889258196413, 'Freddy x Jason')]